In [24]:
import glob
import os
import h5py
import pandas as pd
os.chdir("/home/donghan/DeepLabCut/data/")

filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[200:] 

In [33]:
csvfile

['1034 SI_B, Aug 15, 13 7 49DeepCut_resnet50_ReachingMar12shuffle1_800.h.csv']

In [40]:
import cv2
import os

# Color define, could be different RGB
WHITE = (255,255,255)
BLACK = (0,0,0)

os.chdir("/home/donghan/DeepLabCut/data/")
#Change to your own video data directory

def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass


    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out




def walkingPath(data, filename, outputName, color, displayVideo = False):
    '''
    Graph the walking path of the mouse movement
    '''
    
    head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
    tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
        
    #cap: Read video
    #out: Write video
    video = videoWriter("walking path", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        try:
            for x, y in tail:
                ret,img = cap.read()
                img[x][y] = BLACK
            out.write(img)
            
            # Break the loop when the last frame has been rotated
            if count == frameCnt:
                print (filename, 'successfully ', "added field of view!!!")
                break
            else:
                count += 1
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
                
    


In [ ]:
import time
start = time.time()
walkingPath(data, "1034 SI_B, Aug 15, 13 7 49.mp4", "walkingPathTest.mp4", BLACK)
time.time() - start

In [14]:
import cv2
import numpy as np
image = cv2.imread("/home/donghan/DeepLabCut/data/1.jpg")

image[100:200][100:200] = np.array([0,0,0])

In [30]:
tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
for i, j in tail:
    image[i][j] = (0,0,0)

In [3]:
!pwd

/home/donghan/Neural-Decoding-Project


In [31]:
cv2.imwrite("test.jpg", image)

True